# Race Dashboard



This document describes the requirements and design decisions we will adopt for the Race Dashboard for the Sanofi Asset Efficiency challenge.  This document is intended to provide a detailed explanation of what is to be included on the dashboard, required data sources, and assumptions being made in the design.  It will also show the steps for getting the data into the necessary format.

## Scope/objective

The objective is to be able to display the documented metrics and categories as best suited on a Dashboard presentation for Sanofi staff to be able to access.


## Metrics / categories
The metrics have been mapped into sectors to mimic different sectors of a race track.  The metrics are:
 
 
 
•     Race = 8 Laps = 8 Months   
•     Lap = Monthly Progress   
<br>
<br>

•     **Sector 1 = OEE Improvement   
•     Sector 2 = OEE Variability Improvement   
•     Sector 3 = Stoppage Reduction   
•     Sector 4 = Changeover Improvement**   
• _Sector 5 = Most effective OEE application   
• Sector 6 = Best Innovation   
• Sector 7 = Most consistent OEE improvement progress   
• Sector 8 = Collaboration   
• Sector 9 = Team Spirit_

 
 


A draft impression of a dashboard was produced to create ideas.  This will be followed as much is possible/feasible: 
    
![image](Documents/McLaren2021/Sanofi/raceDashboardMockup.png)

In [19]:
import pandas as pd
import numpy as np
import datetime


# Viz libs
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

# display options
pd.options.display.float_format = "{:.2f}".format


## Read file and cleanse

- QSWeeklyFigures.xlsx should contain the latest data from QS   
- QSDashboard.xlsx has a list of plants/sites taking part, with original target OEE  
- QSWeeklyUnplannedTechLoss.xles should contain the latest data from QS (my unplanned chart by line/week)

#### Cleaning required:   
- OEE % needs converting to numeric, coerce the nulls to nan values
- I think W53-2020 data is bobbings - causes dup indexes for date 2021-01-10 and don't need 2020 data anyway...so dropping it

In [20]:
dir = "C:/Users/mark_/McLaren Technology Group/McLaren Accelerator - Sanofi - Sanofi/Data Analysis/"
output_dir = "C:/Users/mark_/Documents/McLaren2021/Sanofi/"

file = (dir + 'OEE.xlsx')
df_weekly = pd.read_excel(file)
file = (dir + 'QSDashboard.xlsx')
df_dash = pd.read_excel(file)
file = (dir + 'Unplanned_tech_loss.xlsx')
df_techloss = pd.read_excel(file)
file = (dir + 'changeover.xlsx')
df_changeover = pd.read_excel(file)
df_weekly = df_weekly.loc[df_weekly['Week'].str.contains('2021')]
df_techloss = df_techloss.loc[df_techloss['Week'].str.contains('2021')]
df_weekly['OEE %'] = pd.to_numeric(df_weekly['OEE %'], errors='coerce')
df_techloss.rename(columns={'Unplanned losses - %OEE':'Unplanned_tech_loss'}, inplace=True)
df_techloss['Unplanned_tech_loss'] = pd.to_numeric(df_techloss['Unplanned_tech_loss'], errors='coerce')
df_changeover.rename(columns={'Change over losses - %OEE':'Changeover'}, inplace=True)
df_changeover['Changeover'] = pd.to_numeric(df_changeover['Changeover'], errors='coerce')
# don't use their progress figure as it's a static val
# df_dash.rename(columns={'⇗ OEE% progress':'OEE% progress'}, inplace=True)

Get the Pole Q time, and Lando's best Q time from Paul Ricard 2019

In [21]:
import fastf1 as ff1
ff1.Cache.enable_cache('C:/Users/mark_/Documents/McLaren2021/Sanofi/f1cache')
laps = ff1.get_session(2019, 'France', 'Q').load_laps(with_telemetry=True)
best_lando = laps.pick_driver('NOR').pick_fastest()
pole = laps.pick_fastest()

print(best_lando['LapTime'])
print(pole['LapTime'])

core           INFO 	Loading laps for French Grand Prix - Qualifying [v2.1.6]
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
core           INFO 	Loaded data for 20 drivers: ['88', '7', '44', '27', '77', '63', '55', '4', '18', '16', '33', '8', '99', '11', '23', '5', '10', '3', '20', '26']


0 days 00:01:29.418000
0 days 00:01:28.319000


create a datetime from the week number

In [22]:
df_weekly['WeekOfYear'] = pd.to_numeric(df_weekly['Week'].str[1:3])
df_weekly['Year'] = pd.to_numeric(df_weekly['Week'].str[4:])
dates = df_weekly.Year*100+df_weekly.WeekOfYear
df_weekly['Date'] = pd.to_datetime(dates.astype(str) + '0', format='%Y%W%w')
df_weekly.drop(columns=['Year','WeekOfYear'], inplace=True)
df_weekly.head()

,Week,Line,OEE %,Date
11,W01-2021,C2 Packaging Line,0.17,2021-01-10
12,W01-2021,C9 Packaging Line,NaN,2021-01-10
13,W01-2021,GAMMA1,0.41,2021-01-10
14,W01-2021,IMA C80/2,0.51,2021-01-10
15,W01-2021,L18 Packaging Line,0.17,2021-01-10


In [23]:
#merge the 2 dataframes to get the start OEE
df_weekly = df_weekly.merge(df_dash[['Plant','Line', 'OEE  Start point','OEE% Target (2022)']],on='Line')

In [24]:
df_weekly = df_weekly.merge(df_techloss[['Line', 'Week', 'Unplanned_tech_loss']],on=['Line','Week'])

In [25]:
df_weekly = df_weekly.merge(df_changeover[['Line','Week','Changeover']])

#### Start Changeover

Start changeover value isn't provided, so going to calc our own start point using the average changeover for each site in 2021 up to April 2021.  This needs to be done before we drop the early 2021 rows.

This is then merged into the df_weekly dataframe as a loose join.

In [26]:
start_changeover_calc = df_weekly[['Plant','Line','Changeover']][df_weekly['Date'] < '2021-04-30'].groupby(['Plant','Line']).mean().reset_index()
start_changeover_calc.rename(columns={'Changeover':'start_changeover'}, inplace=True)
df_weekly = df_weekly.merge(start_changeover_calc[['Line','start_changeover']])

Turn decimals into percentages before we go any calcs?? Not doing this at the moment as it's useful having similar values for calculating the sector times later.

In [9]:
# df_weekly[['OEE %','OEE  Start point','OEE% Target (2022)','Unplanned_tech_loss','Changeover']] = df_weekly[['OEE %','OEE  Start point','OEE% Target (2022)','Unplanned_tech_loss','Changeover']] * 100

#### Dates for the Asset Challenge

Start Date is going to be fixed as 2021-04-01. Remove all the rows from df_weekly before this date

End Date will move and act as a cutoff before each Race meeting

In [27]:
start_date = '2021-04-01'
df_weekly = df_weekly[df_weekly['Date'] > start_date].sort_values('Date')

end_date = '2021-07-15'
df_weekly = df_weekly[df_weekly['Date'] < end_date].sort_values('Date')

### PCT_CHANGE
Using pct_change python function with periods=4, giving a 4 week (4 previous rows) rolling pct_change figure

- Not sure whether this is required for all of the categories?

In [28]:
df_weekly.sort_values(['Line','Date'], inplace = True)
df_weekly['OEE_pct_chg'] = (df_weekly.groupby('Line')['OEE %']
                                   .apply(pd.Series.pct_change, periods=4))
df_weekly['techloss_pct_chg'] = (df_weekly.groupby('Line')['Unplanned_tech_loss']
                                   .apply(pd.Series.pct_change, periods=4))
df_weekly['Changeover_pct_chg'] = (df_weekly.groupby('Line')['Changeover']
                                   .apply(pd.Series.pct_change, periods=4))
df_weekly.head()

,Week,Line,OEE %,Date,Plant,OEE Start point,OEE% Target (2022),Unplanned_tech_loss,Changeover,start_changeover,OEE_pct_chg,techloss_pct_chg,Changeover_pct_chg
253,W13-2021,AL5 Packaging 1,NaN,2021-04-04,Frankfurt,0.48,0.50,NaN,NaN,0.10,NaN,NaN,NaN
254,W14-2021,AL5 Packaging 1,NaN,2021-04-11,Frankfurt,0.48,0.50,NaN,NaN,0.10,NaN,NaN,NaN
255,W15-2021,AL5 Packaging 1,0.45,2021-04-18,Frankfurt,0.48,0.50,0.31,0.09,0.10,NaN,NaN,NaN
256,W16-2021,AL5 Packaging 1,0.64,2021-04-25,Frankfurt,0.48,0.50,0.14,0.12,0.10,NaN,NaN,NaN
257,W17-2021,AL5 Packaging 1,0.51,2021-05-02,Frankfurt,0.48,0.50,0.28,0.06,0.10,NaN,NaN,NaN


## Standard Deviation
Calculate std_dev and mean on a 4 week rolling basis

Standard deviation is the square root of the variance, so no need to calculate both and have left var out

In [29]:
df_weekly['rolling_std'] = df_weekly.groupby('Line')['OEE %'].apply(lambda x : x.rolling(4,1).agg(np.std))
df_weekly.head(50)

,Week,Line,OEE %,Date,Plant,OEE Start point,OEE% Target (2022),Unplanned_tech_loss,Changeover,start_changeover,OEE_pct_chg,techloss_pct_chg,Changeover_pct_chg,rolling_std
253,W13-2021,AL5 Packaging 1,NaN,2021-04-04,Frankfurt,0.48,0.50,NaN,NaN,0.10,NaN,NaN,NaN,NaN
254,W14-2021,AL5 Packaging 1,NaN,2021-04-11,Frankfurt,0.48,0.50,NaN,NaN,0.10,NaN,NaN,NaN,NaN
255,W15-2021,AL5 Packaging 1,0.45,2021-04-18,Frankfurt,0.48,0.50,0.31,0.09,0.10,NaN,NaN,NaN,NaN
256,W16-2021,AL5 Packaging 1,0.64,2021-04-25,Frankfurt,0.48,0.50,0.14,0.12,0.10,NaN,NaN,NaN,0.14
257,W17-2021,AL5 Packaging 1,0.51,2021-05-02,Frankfurt,0.48,0.50,0.28,0.06,0.10,NaN,NaN,NaN,0.10
258,W18-2021,AL5 Packaging 1,0.44,2021-05-09,Frankfurt,0.48,0.50,0.30,0.09,0.10,NaN,NaN,NaN,0.09
259,W19-2021,AL5 Packaging 1,0.51,2021-05-16,Frankfurt,0.48,0.50,0.23,0.06,0.10,0.13,-0.27,-0.30,0.08
260,W20-2021,AL5 Packaging 1,NaN,2021-05-23,Frankfurt,0.48,0.50,NaN,NaN,0.10,-0.21,0.71,-0.51,0.04
261,W21-2021,AL5 Packaging 1,NaN,2021-05-30,Frankfurt,0.48,0.50,NaN,NaN,0.10,0.00,-0.18,-0.04,0.05
262,W22-2021,AL5 Packaging 1,0.00,2021-06-06,Frankfurt,0.48,0.50,0.00,0.08,0.10,-1.00,-1.00,-0.10,0.36


In [30]:
# df_weekly['OEE% progress'] = df_weekly['OEE %'] - df_weekly['OEE  Start point']

### Calculating Sector times


New instructions 2021-06-11:

The lap time is defined by the sum of the sector scores, with the lowest total equalling the pole time for Ricard.  The following times will be pro-rated against this time (for the total sector score recorded) to give a slightly longer overall lap time.    



Sector [1-4] calculations   
**sector 1**
How much has your OEE increased / decreased?  Sum difference between each week

df_weekly['sector_1'] = df_weekly['OEE_Diff']

OEE_Diff calculation
- Sort values by Line and Date
- Find the difference between each weekly OEE figure
- Fill NaN values from missing OEE figures with the OEE Start Point for that site


**Sector 2** 
How big was your rolling std deviation this period, over the previous 4 weeks std dev?

df_weekly['sector_2'] = df_weekly['rolling_std']

rolling_std = rolling std deviation for past 4 weeks for each site


 
**Sector 3**
We want to reduce Unplanned tech loss (recorded as % of OEE) Unplanned tech loss is calculated within QlikSense but missing values sometimes.  Fill the missing values and then display the average Unplanned tech loss :

df_weekly['sector_3'] = df_weekly['Unplanned_tech_loss']


Populate missing unplanned tech loss:
- Create weekly min/max cols for Unplanned tech loss from any site 
- Merge those columns into df_weekly 
- fill any NaN unplanned tech loss rows with the max OEE calc'd for that week (bigger is worse)

 
**Sector 4**
We're trying to reduce changeover time (recorded as % of OEE).  
Start changeover value isn't provided, so calc our own start point for each Line using the average changeover in 2021 up to 30 April, 2021.
   
start_changeover_calc = df_weekly[['Plant','Line','Changeover']][df_weekly['Date'] < '2021-04-30'].groupby(['Plant','Line']).mean().reset_index()
start_changeover_calc.rename(columns={'Changeover':'start_changeover'}, inplace=True)
df_weekly = df_weekly.merge(start_changeover_calc[['Line','start_changeover']])



df_weekly['sector_4'] = df_weekly['Changeover_rolling_mean']

Changeover_mean = df_weekly.sort_values(by=['Line', 'Date'])[['Line', 'Date', 'Changeover', 'start_changeover']]
Changeover_mean['Changeover_rolling_mean'] = Changeover_mean.groupby('Line')['Changeover'].apply(lambda x : x.rolling(4,1).mean())
df_weekly = df_weekly.merge(Changeover_mean[["Line","Date","Changeover_rolling_mean"]], on=(["Line","Date"]))



**Clean the sectors of NaN before summing them**   
Sometimes, when we haven't got enough information for pct_change calcs, we were getting no values coming through for the lap_time.  We should make sure there is a value in each of the sectors, otherwise there is an unfair advantage by not having data available.  Find all NaN values and replace with the mean for that column(sector)

**Sectors 5 - 9**   
These scores are taken from the Nomination process.  Read in the Nomination s/s, merge any values we find with df_weekly, replace all NaN (missing) values with 0, and reduce the scores we find to 10% of their original value.  This value is then subtracted from the lap_time - so the better you do in the nominations the more your lap_time gets reduced by.

 
**lap_time**
df_weekly['lap_time'] = df_weekly[['sector_1','sector_2','sector_3','sector_4']].sum(axis=1)

In [31]:
file = (dir + 'Nominations Category Scoring.xlsx')
df_nom_sectors = pd.read_excel(file, sheet_name='Nomination scoring', usecols="A:H", parse_dates=['Date'])

In [32]:
df_nom_sectors['Date'] = pd.Series(df_nom_sectors['Date']).fillna(method='ffill')
df_nom_sectors = df_nom_sectors.fillna(0)

df_weekly = df_weekly.merge(df_nom_sectors[['Line','Plant','Date','Best Solution','Best Innovation','Improvement Iterations','Lessons and Sharing','Team Contribution and Spirit']], how='outer', on=['Date','Plant','Line'])
df_weekly

,Week,Line,OEE %,Date,Plant,OEE Start point,OEE% Target (2022),Unplanned_tech_loss,Changeover,start_changeover,OEE_pct_chg,techloss_pct_chg,Changeover_pct_chg,rolling_std,Best Solution,Best Innovation,Improvement Iterations,Lessons and Sharing,Team Contribution and Spirit
0,W13-2021,AL5 Packaging 1,NaN,2021-04-04,Frankfurt,0.48,0.50,NaN,NaN,0.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,W14-2021,AL5 Packaging 1,NaN,2021-04-11,Frankfurt,0.48,0.50,NaN,NaN,0.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,W15-2021,AL5 Packaging 1,0.45,2021-04-18,Frankfurt,0.48,0.50,0.31,0.09,0.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,W16-2021,AL5 Packaging 1,0.64,2021-04-25,Frankfurt,0.48,0.50,0.14,0.12,0.10,NaN,NaN,NaN,0.14,NaN,NaN,NaN,NaN,NaN
4,W17-2021,AL5 Packaging 1,0.51,2021-05-02,Frankfurt,0.48,0.50,0.28,0.06,0.10,NaN,NaN,NaN,0.10,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,NaN,AL5 Packaging 1,NaN,2021-09-30,Frankfurt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,0.00,0.00,0.00
202,NaN,AL6,NaN,2021-09-30,Frankfurt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,0.00,0.00,0.00
203,NaN,M18 Filling,NaN,2021-09-30,Frankfurt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,0.00,0.00,0.00
204,NaN,M21 Filling,NaN,2021-09-30,Frankfurt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,0.00,0.00,0.00


#### Populate missing OEE %
- Find the weekly min/max OEE % from any site   
- Merge those columns into df_weekly   
- fill any NaN with the min OEE we calc'd for that week   

In [33]:
df_weekly_minmax = (df_weekly.assign(Data_Value=df_weekly['OEE %'].abs())
       .groupby(pd.Grouper(key='Date',freq='W'))['OEE %'].agg([('Min' , 'min'), ('Max', 'max')])
       .add_prefix('Week'))
df_weekly_minmax.reset_index(inplace=True)
df_weekly = df_weekly.merge(df_weekly_minmax[['Date','WeekMin','WeekMax']])
df_weekly['OEE %'].fillna(df_weekly.WeekMin, inplace=True)
df_weekly

,Week,Line,OEE %,Date,Plant,OEE Start point,OEE% Target (2022),Unplanned_tech_loss,Changeover,start_changeover,...,techloss_pct_chg,Changeover_pct_chg,rolling_std,Best Solution,Best Innovation,Improvement Iterations,Lessons and Sharing,Team Contribution and Spirit,WeekMin,WeekMax
0,W13-2021,AL5 Packaging 1,0.05,2021-04-04,Frankfurt,0.48,0.50,NaN,NaN,0.10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.05,0.65
1,W13-2021,AL6,0.37,2021-04-04,Frankfurt,0.33,0.45,0.34,0.18,0.11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.05,0.65
2,W13-2021,C2 Packaging Line,0.46,2021-04-04,Maisons-Alfort,0.40,0.47,0.47,0.04,0.05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.05,0.65
3,W13-2021,C9 Packaging Line,0.50,2021-04-04,Maisons-Alfort,0.53,0.53,0.31,0.11,0.03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.05,0.65
4,W13-2021,GAMMA1,0.43,2021-04-04,SCOPPITO,0.42,0.57,0.29,0.21,0.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.05,0.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,W23-2021,SUPPO Packaging Line,0.28,2021-06-13,Lisieux,0.35,0.53,NaN,NaN,0.10,...,inf,inf,0.08,0.00,0.00,0.00,0.00,0.00,0.28,0.68
142,W23-2021,TR200 Packaging Line,0.54,2021-06-13,Lisieux,0.48,0.65,0.22,0.12,0.19,...,-0.02,-0.19,0.12,NaN,NaN,NaN,NaN,NaN,0.28,0.68
143,NaN,LINE 01 - UHLMANN 1880,0.28,2021-06-13,SUZANO,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.00,0.00,0.00,0.00,0.00,0.28,0.68
144,NaN,MEDISEAL PURAN,0.28,2021-06-13,SUZANO,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.00,0.00,0.00,0.00,0.00,0.28,0.68


#### We need the diff between the weekly OEE % figures, and the Weekly Changeover figures
Need something to calculate the OEE Progress and Changeover, otherwise we will have problems when we group and sum values later
- Create OEE_diff with OEE % from groupby of each Line, Week (only 1 row per week, so 'mean' will yield the same)   
- Find the diff between the rows in OEE_Diff for each Line   
- fillNA (first row for each Line) with OEE Start point - should only be needed on the first row for each Line   

repeat same logic for Changeover - there will be more NaN as start_changeover wasn't provided for all.  We populate this later

In [34]:
# this was calculating the wrong Diff - the first row of each site was looking at the previous site for all but the 1st calc
# needed to sort by Line and Date first 

# OEE_Diff = df_weekly.groupby(['Line',pd.Grouper(key='Date',freq='W')])['OEE %'].mean().reset_index()
# OEE_Diff["OEE_Diff"] = OEE_Diff["OEE %"].diff()
# df_weekly = df_weekly.merge(OEE_Diff[["Line","Date","OEE_Diff"]], on=(["Line","Date"]))

# df_weekly['OEE_Diff'].fillna(df_weekly['OEE %'] - df_weekly['OEE  Start point'], inplace=True)
# df_weekly[["Line","Date","OEE %","OEE_Diff"]].head(50).sort_values(by=['Line', 'Date'])

In [35]:
OEE_Diff = df_weekly.sort_values(by=['Line', 'Date'])[['Line','Date','OEE %','OEE  Start point']]
OEE_Diff['OEE_Diff'] = OEE_Diff.groupby('Line')['OEE %'].diff().fillna(df_weekly['OEE  Start point'])
df_weekly = df_weekly.merge(OEE_Diff[["Line","Date","OEE_Diff"]], on=(["Line","Date"]))
df_weekly[["Line","Date","OEE %","OEE_Diff"]].head(50).sort_values(by=['Line', 'Date'])

,Line,Date,OEE %,OEE_Diff
0,AL5 Packaging 1,2021-04-04,0.05,0.48
13,AL5 Packaging 1,2021-04-11,0.35,0.30
26,AL5 Packaging 1,2021-04-18,0.45,0.10
39,AL5 Packaging 1,2021-04-25,0.64,0.19
1,AL6,2021-04-04,0.37,0.33
14,AL6,2021-04-11,0.36,-0.01
27,AL6,2021-04-18,0.33,-0.03
40,AL6,2021-04-25,0.31,-0.02
2,C2 Packaging Line,2021-04-04,0.46,0.40
15,C2 Packaging Line,2021-04-11,0.53,0.07


In [36]:
# Changeover_Diff = df_weekly.groupby(['Line',pd.Grouper(key='Date',freq='W')])['Changeover'].mean().reset_index()
# Changeover_Diff["Changeover_Diff"] = Changeover_diff["Changeover"].diff()
# df_weekly = df_weekly.merge(Changeover_diff[["Line","Date","Changeover_Diff"]], on=(["Line","Date"]))

# df_weekly['Changeover_Diff'].fillna(df_weekly['start_changeover'] - df_weekly['Changeover'], inplace=True)

In [37]:
Changeover_Diff = df_weekly.sort_values(by=['Line', 'Date'])[['Line','Date','Changeover','start_changeover']]
Changeover_Diff['Changeover_Diff'] = Changeover_Diff.groupby('Line')['Changeover'].diff().fillna(df_weekly['start_changeover'] - df_weekly['Changeover'])
df_weekly = df_weekly.merge(Changeover_Diff[["Line","Date","Changeover_Diff"]], on=(["Line","Date"]))

In [38]:
Changeover_mean = df_weekly.sort_values(by=['Line', 'Date'])[['Line','Date','Changeover','start_changeover']]
Changeover_mean['Changeover_rolling_mean'] = Changeover_mean.groupby('Line')['Changeover'].apply(lambda x : x.rolling(4,1).mean())
df_weekly = df_weekly.merge(Changeover_mean[["Line","Date","Changeover_rolling_mean"]], on=(["Line","Date"]))

#### Populate missing Unplanned Tech Loss

- Create weekly min/max cols for Unplanned tech loss from any site   
- Merge those columns into df_weekly   
- fill any NaN rows with the max OEE calc'd for that week   

**this might be flawed!!** 

In [39]:
df_weekly_minmax = (df_weekly.assign(Data_Value=df_weekly['Unplanned_tech_loss'].abs())
       .groupby(pd.Grouper(key='Date',freq='W'))['Unplanned_tech_loss'].agg([('Min' , 'min'), ('Max', 'max')])
       .add_prefix('WeekUTL'))
df_weekly_minmax.reset_index(inplace=True)
df_weekly = df_weekly.merge(df_weekly_minmax[['Date','WeekUTLMin','WeekUTLMax']])
df_weekly['Unplanned_tech_loss'].fillna(df_weekly.WeekUTLMax, inplace=True)
df_weekly

,Week,Line,OEE %,Date,Plant,OEE Start point,OEE% Target (2022),Unplanned_tech_loss,Changeover,start_changeover,...,Improvement Iterations,Lessons and Sharing,Team Contribution and Spirit,WeekMin,WeekMax,OEE_Diff,Changeover_Diff,Changeover_rolling_mean,WeekUTLMin,WeekUTLMax
0,W13-2021,AL5 Packaging 1,0.05,2021-04-04,Frankfurt,0.48,0.50,0.47,NaN,0.10,...,NaN,NaN,NaN,0.05,0.65,0.48,NaN,NaN,0.11,0.47
1,W13-2021,AL6,0.37,2021-04-04,Frankfurt,0.33,0.45,0.34,0.18,0.11,...,NaN,NaN,NaN,0.05,0.65,0.33,-0.08,0.18,0.11,0.47
2,W13-2021,C2 Packaging Line,0.46,2021-04-04,Maisons-Alfort,0.40,0.47,0.47,0.04,0.05,...,NaN,NaN,NaN,0.05,0.65,0.40,0.01,0.04,0.11,0.47
3,W13-2021,C9 Packaging Line,0.50,2021-04-04,Maisons-Alfort,0.53,0.53,0.31,0.11,0.03,...,NaN,NaN,NaN,0.05,0.65,0.53,-0.08,0.11,0.11,0.47
4,W13-2021,GAMMA1,0.43,2021-04-04,SCOPPITO,0.42,0.57,0.29,0.21,0.25,...,NaN,NaN,NaN,0.05,0.65,0.42,0.04,0.21,0.11,0.47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,W23-2021,SUPPO Packaging Line,0.28,2021-06-13,Lisieux,0.35,0.53,0.43,NaN,0.10,...,0.00,0.00,0.00,0.28,0.68,-0.10,NaN,0.05,0.10,0.43
142,W23-2021,TR200 Packaging Line,0.54,2021-06-13,Lisieux,0.48,0.65,0.22,0.12,0.19,...,NaN,NaN,NaN,0.28,0.68,-0.23,0.06,0.13,0.10,0.43
143,NaN,LINE 01 - UHLMANN 1880,0.28,2021-06-13,SUZANO,NaN,NaN,0.43,NaN,NaN,...,0.00,0.00,0.00,0.28,0.68,NaN,NaN,NaN,0.10,0.43
144,NaN,MEDISEAL PURAN,0.28,2021-06-13,SUZANO,NaN,NaN,0.43,NaN,NaN,...,0.00,0.00,0.00,0.28,0.68,NaN,NaN,NaN,0.10,0.43


#### Populate missing Changeover 

In [40]:
df_weekly_minmax = (df_weekly.assign(Data_Value=df_weekly['Changeover'].abs())
       .groupby(pd.Grouper(key='Date',freq='W'))['Changeover'].agg([('Min' , 'min'), ('Max', 'max')])
       .add_prefix('WeekChangeover'))
df_weekly_minmax.reset_index(inplace=True)
df_weekly = df_weekly.merge(df_weekly_minmax[['Date','WeekChangeoverMin','WeekChangeoverMax']])
df_weekly['Changeover'].fillna(df_weekly.WeekChangeoverMax, inplace=True)
df_weekly

,Week,Line,OEE %,Date,Plant,OEE Start point,OEE% Target (2022),Unplanned_tech_loss,Changeover,start_changeover,...,Team Contribution and Spirit,WeekMin,WeekMax,OEE_Diff,Changeover_Diff,Changeover_rolling_mean,WeekUTLMin,WeekUTLMax,WeekChangeoverMin,WeekChangeoverMax
0,W13-2021,AL5 Packaging 1,0.05,2021-04-04,Frankfurt,0.48,0.50,0.47,0.79,0.10,...,NaN,0.05,0.65,0.48,NaN,NaN,0.11,0.47,0.04,0.79
1,W13-2021,AL6,0.37,2021-04-04,Frankfurt,0.33,0.45,0.34,0.18,0.11,...,NaN,0.05,0.65,0.33,-0.08,0.18,0.11,0.47,0.04,0.79
2,W13-2021,C2 Packaging Line,0.46,2021-04-04,Maisons-Alfort,0.40,0.47,0.47,0.04,0.05,...,NaN,0.05,0.65,0.40,0.01,0.04,0.11,0.47,0.04,0.79
3,W13-2021,C9 Packaging Line,0.50,2021-04-04,Maisons-Alfort,0.53,0.53,0.31,0.11,0.03,...,NaN,0.05,0.65,0.53,-0.08,0.11,0.11,0.47,0.04,0.79
4,W13-2021,GAMMA1,0.43,2021-04-04,SCOPPITO,0.42,0.57,0.29,0.21,0.25,...,NaN,0.05,0.65,0.42,0.04,0.21,0.11,0.47,0.04,0.79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,W23-2021,SUPPO Packaging Line,0.28,2021-06-13,Lisieux,0.35,0.53,0.43,0.28,0.10,...,0.00,0.28,0.68,-0.10,NaN,0.05,0.10,0.43,0.00,0.28
142,W23-2021,TR200 Packaging Line,0.54,2021-06-13,Lisieux,0.48,0.65,0.22,0.12,0.19,...,NaN,0.28,0.68,-0.23,0.06,0.13,0.10,0.43,0.00,0.28
143,NaN,LINE 01 - UHLMANN 1880,0.28,2021-06-13,SUZANO,NaN,NaN,0.43,0.28,NaN,...,0.00,0.28,0.68,NaN,NaN,NaN,0.10,0.43,0.00,0.28
144,NaN,MEDISEAL PURAN,0.28,2021-06-13,SUZANO,NaN,NaN,0.43,0.28,NaN,...,0.00,0.28,0.68,NaN,NaN,NaN,0.10,0.43,0.00,0.28


#### Populate missing start_changeover (NaN)
If we've got this far and you still don't have a start_changeover, then you're a new site and can have this week's Changeover value

These rows still have NaN Changeover values

In [41]:
df_weekly[['Line','Date','Changeover','start_changeover']][df_weekly['Changeover'].isna()]

,Line,Date,Changeover,start_changeover


In [42]:
df_weekly.start_changeover.fillna(df_weekly.Changeover, inplace=True)
df_weekly['OEE  Start point'].fillna(df_weekly['OEE %'], inplace=True)
df_weekly['OEE% Target (2022)'].fillna(0.65, inplace=True)

#### Sector times

In [43]:
# df_weekly['sector_1'] = (df_weekly['WeekMax'] - df_weekly['OEE %'])
# df_weekly['sector_1'] = (df_weekly['OEE  Start point'] - df_weekly['OEE %'])
df_weekly['sector_1'] = df_weekly['OEE_Diff']
df_weekly['sector_2'] = df_weekly['rolling_std']
df_weekly['sector_3'] = df_weekly['Unplanned_tech_loss']
# df_weekly['sector_4'] = (df_weekly['start_changeover'] - df_weekly['Changeover'])
# df_weekly['sector_4'] = (df_weekly['Changeover'] - df_weekly['start_changeover'])
df_weekly['sector_4'] = df_weekly['Changeover_rolling_mean']
# take 10% of the sector5-9 scores 
df_weekly[['sector_5','sector_6','sector_7','sector_8','sector_9']] = df_weekly[['Best Solution','Best Innovation','Improvement Iterations','Lessons and Sharing','Team Contribution and Spirit']] * -0.1
df_weekly[['sector_5','sector_6','sector_7','sector_8','sector_9']] = df_weekly[['sector_5','sector_6','sector_7','sector_8','sector_9']].fillna(0)

# we'll use these in the absence of values for a sector
df_weekly[['sector_1','sector_2','sector_3','sector_4']] = df_weekly[['sector_1','sector_2','sector_3','sector_4']].fillna(df_weekly[['sector_1','sector_2','sector_3','sector_4']].mean())

#this will sum and handle the NaN
df_weekly['lap_time'] = df_weekly[['sector_1','sector_2','sector_3','sector_4','sector_5','sector_6','sector_7','sector_8','sector_9']].sum(axis=1)

# now add the pole['Laptime'] from fastf1 to the lap_time adjustment we've created
# just use 88 secs rather than playing with timedeltas for now
# df_weekly['lap_time'] = pole['LapTime'] + pd.to_timedelta(df_weekly['lap_time'], unit='S')
# df_weekly['lap_time'] = 88 + df_weekly['lap_time']
df_weekly.groupby(['Line', pd.Grouper(key='Date', freq='W')])['lap_time'].sum()
# print (df_weekly['sector_1_time'] , df_weekly['sector_2_time'] , df_weekly['sector_3_time'], df_weekly['sector_4_time'])

Line                  Date      
AL5 Packaging 1       2021-04-04   1.21
                      2021-04-11   0.96
                      2021-04-18   0.61
                      2021-04-25   0.57
                      2021-05-02   0.33
                                   ... 
TR200 Packaging Line  2021-05-16   0.34
                      2021-05-23   0.43
                      2021-05-30   0.37
                      2021-06-06   0.67
                      2021-06-13   0.24
Name: lap_time, Length: 146, dtype: float64

#### Write out df_weekly to excel

In [45]:
df_weekly.to_excel(output_dir + "df_weekly_with_calcs.xlsx")

#### Monthly Calcs

Repeat the process for a df_monthly spreadsheet.  We will use this for calculating the Leader board  

In [46]:
# df_monthly = df_weekly.groupby([pd.Grouper(key='Date',freq='M'),'Line'])[['start_changeover','OEE  Start point','OEE %','Unplanned_tech_loss','Changeover','rolling_std','techloss_pct_chg','Changeover_pct_chg']].mean().reset_index()
df_monthly = df_weekly.groupby([pd.Grouper(key='Date',freq='M'),'Line']).lap_time.sum().reset_index()
# df_monthly.set_index('date', inplace=True)
df_monthly[df_monthly['Line'].str.contains('UHL')]

,Date,Line,lap_time
35,2021-06-30,LINE 01 - UHLMANN 1880,0.75


In [47]:
# df_monthly_minmax = (df_weekly.assign(Data_Value=df_weekly['OEE %'].abs())
#        .groupby(pd.Grouper(key='Date',freq='M'))['OEE %'].agg([('Min' , 'min'), ('Max', 'max')])
#        .add_prefix('Month'))
# df_monthly_minmax.reset_index(inplace=True)
# df_monthly = df_monthly.merge(df_monthly_minmax[['Date','MonthMin','MonthMax']])
df_monthly['lap_time'] = df_monthly['lap_time'] + 88
df_monthly

,Date,Line,lap_time
0,2021-04-30,AL5 Packaging 1,91.34
1,2021-04-30,AL6,90.71
2,2021-04-30,C2 Packaging Line,90.67
3,2021-04-30,C9 Packaging Line,90.59
4,2021-04-30,GAMMA1,90.49
5,2021-04-30,IMA C80/2,90.09
6,2021-04-30,L18 Packaging Line,89.96
7,2021-04-30,L25 Packaging Line,90.11
8,2021-04-30,M18 Filling,92.45
9,2021-04-30,M21 Filling,91.82


### Leader board table

In [48]:
pivot = df_monthly[df_monthly['Date'] < end_date].pivot(index='Line', columns='Date', values='lap_time')
pivot.reset_index(inplace=True)
# pivot creates NaN for rows with no monthly data - fix that below for cols 2 and 3
pivot[pivot.columns[1]].fillna(pivot[pivot.columns[2]].max(), inplace=True)
pivot[pivot.columns[2]].fillna(pivot[pivot.columns[3]].max(), inplace=True)

pivot['race_time'] = pivot.sum(axis=1)
pivot = pivot.merge(df_dash[['Plant','Line']], on='Line')
pivot.sort_values('race_time', inplace=True)
pivot['position'] = np.arange(1,len(pivot) + 1)
pivot['gap_to_leader'] = pivot['race_time'] - pivot['race_time'].iloc[0]

pivot['prev_race_time'] = pivot[pivot.columns[2]] + pivot[pivot.columns[3]]
pivot.sort_values('prev_race_time', inplace=True)
pivot['prev_position'] = np.arange(1,len(pivot) + 1)
pivot['Gain/Loss'] = pivot.prev_position - pivot.position
pivot.sort_values('race_time', inplace=True)
pivot['interval'] = pivot.race_time.diff()
pivot = pivot.merge(df_dash[['Line','OEE  Start point', '⇗ OEE% progress', 'OEE% Target (2022)']], on='Line')
pivot

,Line,2021-04-30 00:00:00,2021-05-31 00:00:00,2021-06-30 00:00:00,race_time,Plant,position,gap_to_leader,prev_race_time,prev_position,Gain/Loss,interval,OEE Start point,⇗ OEE% progress,OEE% Target (2022)
0,GAMMA1,90.49,90.28,67.47,248.23,SCOPPITO,1,0.00,157.75,1,0,NaN,0.42,0.09,0.57
1,IMA C80/2,90.09,90.13,74.46,254.67,SCOPPITO,2,6.44,164.59,2,0,6.44,0.45,0.04,0.58
2,AL6,90.71,90.59,83.09,264.40,Frankfurt,3,16.16,173.69,3,0,9.73,0.33,0.08,0.45
3,M22 Filling,90.50,90.49,83.66,264.65,Frankfurt,4,16.41,174.15,4,0,0.25,0.53,0.12,0.65
4,M21 Filling,91.82,92.02,83.85,267.69,Frankfurt,5,19.46,175.87,6,1,3.04,0.60,0.02,0.65
5,M18 Filling,92.45,90.49,85.05,267.99,Frankfurt,6,19.76,175.55,5,-1,0.30,0.44,-0.01,0.65
6,L18 Packaging Line,89.96,89.85,88.67,268.49,Tours,7,20.26,178.53,9,2,0.50,0.38,0.09,0.55
7,TR200 Packaging Line,89.98,89.98,88.90,268.86,Lisieux,8,20.63,178.88,10,2,0.38,0.48,0.12,0.65
8,SUPPO Packaging Line,89.73,90.40,88.86,268.99,Lisieux,9,20.75,179.26,11,2,0.12,0.35,-0.01,0.53
9,L25 Packaging Line,90.11,90.80,88.83,269.74,Tours,10,21.51,179.63,13,3,0.76,0.35,0.00,0.48


#### write this out for tableau

In [ ]:
pivot.to_csv(output_dir + "leaderboard.csv")

# Produce Dashboard charts

Let's color the table output and produce some charts

In [ ]:
# not using this at the moment - setting it will make the charts fail later as there is no date col
# df_race.set_index('date', inplace=True)

In [ ]:
df_weekly.pivot(index='Date', columns='Line', values='rolling_std')

In [ ]:
# Initialize the matplotlib figure

fig, axs = plt.subplots(6, figsize=(16, 30))
plt.grid(True, color='gray')
# plt.plot(df_race[['Line','Monthly OEE % Variability']],label='Variability')

# plt.subplot(2, 1, 1)

# data 
pivot_data = df_weekly.pivot(index='Date', columns='Line')


# axs 0
current_axs = 0
df_weekly.pivot(index='Date', columns='Line', values='OEE %').plot(ax=axs[current_axs])
# df_race.pivot(index='date', columns='Line', values='OEE Improvement %').plot(ax=ax)
# plt.plot(df_race['OEE Improvement %'],label='OEE Improvement %')
axs[current_axs].set_facecolor('black')
axs[current_axs].set_ylabel('OEE %')
axs[current_axs].set_xlabel('')
axs[current_axs].legend(labelcolor='linecolor')

# axs 1
current_axs = 1
df_weekly.pivot(index='Date', columns='Line', values='rolling_std').plot(ax=axs[current_axs])
# df_race.pivot(index='date', columns='Line', values='OEE Improvement %').plot(ax=ax)
# plt.plot(df_race['OEE Improvement %'],label='OEE Improvement %')
axs[current_axs].set_facecolor('black')
axs[current_axs].set_ylabel('rolling_std')
axs[current_axs].set_xlabel('')
axs[current_axs].legend(labelcolor='linecolor')


# axs = 2
current_axs = 2
df_weekly.pivot(index='Date', columns='Line', values='OEE_pct_chg').plot(ax=axs[current_axs])
# df_race.pivot(index='date', columns='Line', values='OEE Improvement %').plot(ax=ax)
# plt.plot(df_race['OEE Improvement %'],label='OEE Improvement %')
axs[current_axs].set_facecolor('black')
axs[current_axs].set_ylabel('OEE_pct_change')
axs[current_axs].set_xlabel('')
axs[current_axs].legend(labelcolor='linecolor')


# axs = 3
current_axs = 3
df_weekly.pivot(index='Date', columns='Line', values='Unplanned_tech_loss').plot(ax=axs[current_axs])
# df_race.pivot(index='date', columns='Line', values='OEE Improvement %').plot(ax=ax)
# plt.plot(df_race['OEE Improvement %'],label='OEE Improvement %')
axs[current_axs].set_facecolor('black')
axs[current_axs].set_ylabel('Unplanned_tech_loss')
axs[current_axs].set_xlabel('')
axs[current_axs].legend(labelcolor='linecolor')

# axs = 4
# current_axs = 4
# df_weekly.pivot(index='date', columns='Line', values='rolling_mean').plot(ax=axs[current_axs])
# # df_race.pivot(index='date', columns='Line', values='OEE Improvement %').plot(ax=ax)
# # plt.plot(df_race['OEE Improvement %'],label='OEE Improvement %')
# axs[current_axs].set_facecolor('black')
# axs[current_axs].set_ylabel('rolling_mean')
# axs[current_axs].set_xlabel('')



# axs = 4
current_axs = 4
df_weekly.pivot(index='Date', columns='Line', values='Changeover').plot(ax=axs[current_axs])
# df_race.pivot(index='date', columns='Line', values='OEE Improvement %').plot(ax=ax)
# plt.plot(df_race['OEE Improvement %'],label='OEE Improvement %')
axs[current_axs].set_facecolor('black')
axs[current_axs].set_ylabel('Changeover')
axs[current_axs].set_xlabel('')
axs[current_axs].legend(labelcolor='linecolor')

# axs = 5
current_axs = 5
df_weekly.pivot(index='Date', columns='Line', values='lap_time').plot(ax=axs[current_axs])
# df_race.pivot(index='date', columns='Line', values='OEE Improvement %').plot(ax=ax)
# plt.plot(df_race['OEE Improvement %'],label='OEE Improvement %')
axs[current_axs].set_facecolor('black')
axs[current_axs].set_ylabel('lap_time')
axs[current_axs].set_xlabel('')
axs[current_axs].legend(labelcolor='linecolor')

# draw gridlines
axs[0].grid(which='both', linestyle='--', color='gray')
axs[1].grid(which='both', linestyle='--', color='gray')
axs[2].grid(which='both', linestyle='--', color='gray')
axs[2].set_ylim(0,5)
axs[3].grid(which='both', linestyle='--', color='gray')
# axs[4].grid(which='both', linestyle='--', color='gray')
axs[4].grid(which='both', linestyle='--', color='gray')
axs[5].grid(which='both', linestyle='--', color='gray')


# axs[1].text(0.50, 0.50, 'Race Trends', fontsize=35, ha='center', color='orange', style='italic', weight='bold',
#              bbox=dict(facecolor='gold', alpha=0.5, pad=10, boxstyle='round, pad=.9'))
axs[0].text(0.25, 0.8, 'OEE %', color='white', fontsize=32, bbox=dict(facecolor='orange', boxstyle='round'), horizontalalignment='left', verticalalignment='center', transform=axs[0].transAxes)
axs[1].text(0.25, 0.8, 'OEE_std_dev', color='white', fontsize=32, bbox=dict(facecolor='orange', boxstyle='round'), horizontalalignment='left', verticalalignment='center', transform=axs[1].transAxes)
axs[2].text(0.25, 0.8, 'pct_change', color='white', fontsize=32, bbox=dict(facecolor='orange', boxstyle='round'), horizontalalignment='left', verticalalignment='center', transform=axs[2].transAxes)
axs[3].text(0.25, 0.8, 'Unplanned_tech_loss', color='white', fontsize=32, bbox=dict(facecolor='orange', boxstyle='round'), horizontalalignment='left', verticalalignment='center', transform=axs[3].transAxes)
# axs[4].text(0.25, 0.8, 'rolling_mean', color='white', fontsize=32, bbox=dict(facecolor='orange', boxstyle='round'), horizontalalignment='left', verticalalignment='center', transform=axs[4].transAxes)
axs[4].text(0.25, 0.8, 'Changeover', color='white', fontsize=32, bbox=dict(facecolor='orange', boxstyle='round'), horizontalalignment='left', verticalalignment='center', transform=axs[4].transAxes)
axs[5].text(0.25, 0.8, 'lap_time', color='white', fontsize=32, bbox=dict(facecolor='orange', boxstyle='round'), horizontalalignment='left', verticalalignment='center', transform=axs[5].transAxes)

# plt.savefig("C:/Users/mark_/Documents/McLaren2021/Sanofi/AL6_Trends.png")

#### Produce the output
Write df_weekly to excel.  This is the summarised data with variance and improvement calcs
Write plot to PDF for importing somewhere

In [ ]:
df_weekly.to_excel(dir + "df_weekly_with_calcs.xlsx")
# not sure I need monthly calcs anymore
# df_monthly.to_excel(dir + "df_monthly.xlsx")

plt.savefig(dir + "raceTrends.png")


### Build the race track

In [ ]:
monthly_race_times = df_weekly[['Date','Plant','Line','lap_time']].groupby(['Line',pd.Grouper(key='Date', freq='W')]).sum().reset_index()
monthly_race_times = df_weekly[['Plant','Line','lap_time']][df_weekly['Date'] == '2021-06-13']
monthly_race_times
monthly_race_times['norm_lap_time']=(monthly_race_times['lap_time']-monthly_race_times['lap_time'].min())/(monthly_race_times['lap_time'].max()-monthly_race_times['lap_time'].min())
monthly_race_times

In [ ]:
tel = pole.telemetry
tel

In [ ]:
#best_lando=best_lando.add_distance()
#best_lando=best_lando.add_relative_distance()
pos = pole.get_pos_data()

In [ ]:
pos = pd.merge(pos, tel[['Time','Distance','Source','RelativeDistance']], how='left', left_on=['Time', 'Source'], right_on=['Time','Source'])

In [ ]:
for i, row in pivot.iterrows():
    xy_lookup = int(row[8] * -1)
    print (xy_lookup)

In [ ]:
pivot.head()

In [60]:
df_monthly[df_monthly['Line'].str.contains('AL6')]

,Date,Line,lap_time
1,2021-04-30,AL6,90.71
14,2021-05-31,AL6,90.59
27,2021-06-30,AL6,83.09


In [ ]:
df_monthly[['Date','OEE_pct_chg','OEE %','OEE% progress']][df_monthly['Line'].str.contains('L25')]

In [59]:
df_weekly[['Week','Line','Changeover_Diff','Changeover_rolling_mean']][df_weekly['Line'].str.contains('AL6')]

,Week,Line,Changeover_Diff,Changeover_rolling_mean
1,W13-2021,AL6,-0.08,0.18
14,W14-2021,AL6,-0.08,0.14
27,W15-2021,AL6,-0.04,0.11
40,W16-2021,AL6,0.03,0.11
53,W17-2021,AL6,-0.01,0.08
66,W18-2021,AL6,0.03,0.08
79,W19-2021,AL6,-0.01,0.09
92,W20-2021,AL6,-0.05,0.08
105,W21-2021,AL6,0.07,0.09
118,W22-2021,AL6,-0.03,0.08


In [61]:
df_weekly[['Date','Plant','Line','lap_time']].groupby(['Line',pd.Grouper(key='Date', freq='W')]).sum().reset_index()

,Line,Date,lap_time
0,AL5 Packaging 1,2021-04-04,1.21
1,AL5 Packaging 1,2021-04-11,0.96
2,AL5 Packaging 1,2021-04-18,0.61
3,AL5 Packaging 1,2021-04-25,0.57
4,AL5 Packaging 1,2021-05-02,0.33
...,...,...,...
141,TR200 Packaging Line,2021-05-16,0.34
142,TR200 Packaging Line,2021-05-23,0.43
143,TR200 Packaging Line,2021-05-30,0.37
144,TR200 Packaging Line,2021-06-06,0.67


In [62]:
df_weekly[['Plant','Line','lap_time']][df_weekly['Date'] == '2021-06-13']

,Plant,Line,lap_time
130,Frankfurt,AL5 Packaging 1,0.87
131,Frankfurt,AL6,-5.46
132,Maisons-Alfort,C2 Packaging Line,0.58
133,Maisons-Alfort,C9 Packaging Line,0.57
134,SCOPPITO,GAMMA1,-21.11
135,SCOPPITO,IMA C80/2,-14.06
136,Tours,L18 Packaging Line,0.24
137,Tours,L25 Packaging Line,0.45
138,Frankfurt,M18 Filling,-4.24
139,Frankfurt,M21 Filling,-4.83


In [63]:
df_weekly

,Week,Line,OEE %,Date,Plant,OEE Start point,OEE% Target (2022),Unplanned_tech_loss,Changeover,start_changeover,...,sector_1,sector_2,sector_3,sector_4,sector_5,sector_6,sector_7,sector_8,sector_9,lap_time
0,W13-2021,AL5 Packaging 1,0.05,2021-04-04,Frankfurt,0.48,0.50,0.47,0.79,0.10,...,0.48,0.10,0.47,0.16,0.00,0.00,0.00,0.00,0.00,1.21
1,W13-2021,AL6,0.37,2021-04-04,Frankfurt,0.33,0.45,0.34,0.18,0.11,...,0.33,0.10,0.34,0.18,0.00,0.00,0.00,0.00,0.00,0.96
2,W13-2021,C2 Packaging Line,0.46,2021-04-04,Maisons-Alfort,0.40,0.47,0.47,0.04,0.05,...,0.40,0.10,0.47,0.04,0.00,0.00,0.00,0.00,0.00,1.01
3,W13-2021,C9 Packaging Line,0.50,2021-04-04,Maisons-Alfort,0.53,0.53,0.31,0.11,0.03,...,0.53,0.10,0.31,0.11,0.00,0.00,0.00,0.00,0.00,1.04
4,W13-2021,GAMMA1,0.43,2021-04-04,SCOPPITO,0.42,0.57,0.29,0.21,0.25,...,0.42,0.10,0.29,0.21,0.00,0.00,0.00,0.00,0.00,1.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,W23-2021,SUPPO Packaging Line,0.28,2021-06-13,Lisieux,0.35,0.53,0.43,0.28,0.10,...,-0.10,0.08,0.43,0.05,-0.00,-0.00,-0.00,-0.00,-0.00,0.47
142,W23-2021,TR200 Packaging Line,0.54,2021-06-13,Lisieux,0.48,0.65,0.22,0.12,0.19,...,-0.23,0.12,0.22,0.13,0.00,0.00,0.00,0.00,0.00,0.24
143,NaN,LINE 01 - UHLMANN 1880,0.28,2021-06-13,SUZANO,0.28,0.65,0.43,0.28,0.28,...,0.05,0.10,0.43,0.16,-0.00,-0.00,-0.00,-0.00,-0.00,0.75
144,NaN,MEDISEAL PURAN,0.28,2021-06-13,SUZANO,0.28,0.65,0.43,0.28,0.28,...,0.05,0.10,0.43,0.16,-0.00,-0.00,-0.00,-0.00,-0.00,0.75
